# Importando as bibliotecas...

In [1]:
import numpy as np
import math as mt
import sympy as sp
import inspect
import re 
import altair as alt
from vega_datasets import data
import pandas as pd
from numpy import cos, sin, exp
import sys
from __future__ import division 
import random
import time

# Capítulo 4 - Solução de sistemas lineares

In [2]:
def gerarMatriz(dimensao: int, densidade_matriz: float):
    #Encontra a quantidade de valores não-nulos na matriz
    num_nao_nulo = int((dimensao**2 )* densidade_matriz)

    #Gerar indices aleatorios
    linha_indices = [random.randint(0, dimensao-1) for _ in range(num_nao_nulo)]
    coluna_indices = [random.randint(0, dimensao-1) for _ in range(num_nao_nulo)]

    #Gera valores aleatorios para os valores não nulos
    valores = [random.randint(1,10) for _ in range(num_nao_nulo)]

    #Gera a matriz esparsa
    esparsa_matriz = [[0 for _ in range(dimensao)] for _ in range(dimensao)]
    for i in range(num_nao_nulo):
        linha = linha_indices[i]
        coluna = coluna_indices[i]
        val = valores[i]
        esparsa_matriz[linha][coluna] = val
    
    return esparsa_matriz

dataf = pd.DataFrame(gerarMatriz(10, 1))

dataf


,0,1,2,3,4,5,6,7,8,9
0,2,0,0,7,10,1,9,0,0,6
1,8,1,8,0,4,0,8,9,0,10
2,0,9,8,3,0,6,2,6,0,4
3,2,10,0,0,0,0,0,6,10,0
4,0,0,1,10,0,10,2,6,6,3
5,0,10,0,8,5,8,6,8,2,1
6,0,1,3,0,10,0,3,6,0,0
7,0,2,0,8,10,0,0,0,6,1
8,10,7,7,5,5,4,2,3,0,7
9,9,2,6,8,0,3,8,8,9,0


In [3]:
def solucaoSuperior(data):
    coluna=len(data[0])-1

    resposta = []
    for linha in range(len(data)-1,-1, -1):
        if(linha == len(data)-1):
            
            aux = data[linha][coluna]/data[linha][linha]
            resposta.append(aux)

        else:

            auxVaAnterior = 0
            tempsoma = 0
            for soma in range(coluna-1, linha, -1): 
                
                tempsoma += data[linha][soma]*resposta[auxVaAnterior]
                auxVaAnterior+=1
        
            aux = (data[linha][coluna] - tempsoma)/data[linha][linha]
            resposta.append(aux)
    return resposta


def solucaoInferior(data):
    coluna=len(data[0])-1

    resposta = []
    for linha in range(len(data)):
        if(linha == 0):
            #print(data[linha][coluna],data[linha][linha])
            aux = data[linha][coluna]/data[linha][linha]
            resposta.append(aux)

        else:

            auxVaAnterior = 0
            tempsoma = 0
            for soma in range(linha):
                
                tempsoma += data[linha][soma]*resposta[auxVaAnterior]
                auxVaAnterior+=1
        
            aux = (data[linha][coluna] - tempsoma)/data[linha][linha]
            resposta.append(aux)
    return resposta

### Atividade 4

* Implemente todas as técnicas de solução de sistemas
lineares vistas em aula e faça uma comparação entre elas
para as seguintes situações:
    * n = [10,1000,2000,3000, … , 20000]
    * Sistemas esparsos (contém muitos zeros na matriz A) e sistemas densos (nenhum zero na matriz A)
    

* Gere sistemas lineares aleatórios que atendam essas
condições

* Mostre um gráfico para o tempo de execução e um com o
número de iterações
    * As linhas devem ser os algoritmos
    * Um gráfico para sistemas esparsos e outro para os densos

### Eliminação Gaussiana (escalonamento)
* método para resolver sistemas lineares
* consiste em manipular o sistema através de determinadas operações elementares, transformando a matriz estendida do sistema em uma matriz triangular (chamada de matriz escalonada do sistema)

In [4]:
def trianguloSuperior(dimensao, data):
  aux = 0
  aux2 = 0
  for i in range(dimensao): 
    aux += i

  for linha in range(dimensao):
    for coluna in range(dimensao):
      if(linha == coluna): break
      elif(data[linha][coluna]==0): aux2 += 1
  
  return True if aux == aux2 else False 

def maiorPivo(data, coluna):
  maior = [0,0]
  maior[0] = data[coluna][coluna]
  maior[1] = coluna

  for linha in range(coluna, len(data)):
    if data[linha][coluna] > maior[0]: 
      maior[0] = data[linha][coluna]
      maior[1] = linha
  return maior[1]

In [5]:
def pivoteamento(iteracao, data, fatores = []):

  auxpivo = iteracao

  if(data[iteracao][iteracao] != 0):
    pivo = data[iteracao][iteracao]
    aux=1

    for linha in range(iteracao+1,len(data)):
      fator = data[linha][iteracao]/pivo
      fatores.append(fator)
      for coluna in range(len(data[0])): 
        data[linha][coluna] = round((data[linha][coluna] - fator*data[linha-aux][coluna]), 4) 

      aux+=1
    return 1
  else:
    while(data[iteracao][iteracao] == 0 and auxpivo != len(data)):
      auxpivo+=1
      temp = data[iteracao].copy()
      data[iteracao] = data[auxpivo].copy()
      data[auxpivo] = temp
      
    return 0



def pivoteamentoParcial(iteracao, data, fatores=[]):

  maiorlinha = maiorPivo(data, iteracao)

  if(data[iteracao][iteracao] != 0 and iteracao == maiorlinha):
    
    pivo = data[iteracao][iteracao]
    aux=1

    for linha in range(iteracao+1,len(data)):
      fator = data[linha][iteracao]/pivo
      fatores.append(fator)
      for coluna in range(len(data[0])): 
        data[linha][coluna] = round((data[linha][coluna] - fator*data[linha-aux][coluna]), 4) 

      aux+=1
    return 1

    
  else:

    temp = data[iteracao].copy()
    data[iteracao] = data[maiorlinha].copy()
    data[maiorlinha] = temp
      
    return 0
      

In [6]:
#eliminação gaussiana com pivoteamento parcial:

#1. definindo a matriz que irei trabalhar

data = [[1,1,1,0],[0,1,10,-48],[1,10,1,25]]

#2. executando o algoritmo de pivoteamento até conseguir um triangulo superior
def eliminacaoGaussianaParcial(data):
  iteracao = 0 

  while(iteracao < len(data)):
    if trianguloSuperior(len(data), data) == False:
      iteracao += pivoteamentoParcial(iteracao, data)
      
    else: break
    

  #3. Substituindo e resolvendo a equação
  resposta = solucaoSuperior(data)
  

  dataf = pd.DataFrame(data)  
  return dataf, iteracao, resposta

dataf, iteracao, resposta=eliminacaoGaussianaParcial(data)
print("Data:\n {}\n\nIteracao:{}\n\nResultado:{}\n ".format(dataf, iteracao, resposta))

Data:
      0    1     2        3
0  1.0  1.0   1.0   0.0000
1  0.0  9.0   0.0  25.0000
2  0.0  0.0  10.0 -50.7778

Iteracao:2

Resultado:[-5.07778, 2.7777777777777777, 2.300002222222222]
 


In [7]:
#eliminação gaussiana:

#1. definindo a matriz que irei trabalhar

data = [[1,1,1,1],[4,4,2,2],[2,1,-1,0]]

#2. executando o algoritmo de pivoteamento até conseguir um triangulo superior
def eliminacaoGaussiana(data):
  iteracao = 0 

  while(iteracao < len(data)):
    if trianguloSuperior(len(data), data) == False:
      iteracao += pivoteamento(iteracao, data)
      
    else: break
    

  #3. Substituindo e resolvendo a equação
  resposta = solucaoSuperior(data)
  dataf = pd.DataFrame(data)  
  return dataf, iteracao, resposta

dataf, iteracao, resposta = eliminacaoGaussiana(data)
print("Data:\n {}\n\nIteracao:{}\n\nResultado:{}\n ".format(dataf, iteracao, resposta))


Data:
      0    1    2    3
0  1.0  1.0  1.0  1.0
1  0.0 -1.0 -3.0 -2.0
2  0.0  0.0 -2.0 -2.0

Iteracao:1

Resultado:[1.0, -1.0, 1.0]
 


### Decomposição LU (Lower Upper)

In [8]:
def menoresPrincipais(data):

    for iteracao in range(len(data)):
        auxM = []
        for linha in range(iteracao+1):
            auxL = []
            for coluna in range(iteracao+1):
                auxL.append(data[linha][coluna])
            #print("Lista:",auxL)  
            auxM.append(auxL)
        

        if np.linalg.det(auxM) == 0: return False
        
    return True 

def identidade(dimensao):
    auxm = []
    for linha in range(dimensao):
        auxl = []
        for coluna in range(dimensao):
            if(linha == coluna):
                auxl.append(1)
            else:
                auxl.append(0)
        auxm.append(auxl)
    return auxm

def metodoLU(A):
    if menoresPrincipais(data) == True:
        U = data.copy()
        linha = len(A)
        L = identidade(linha)
        for j in range(linha-1):
            for i in range(j+1, linha):
                L[i][j] = U[i][j]/U[j][j]  
                for k in range(j+1, linha):
                    U[i][k] = U[i][k] - L[i][j]*U[j][k]
                U[i][j] = 0 
        return L, U
        
    else: print("A matriz não tem decomposição LU.")

In [9]:


data = [[1,1,1],[2,1,-1],[2,-1,1]]
b = [-2, 1, 3]


def solucaoLU(data, b):    
   
    L, U = metodoLU(data)
    
    #1. solucao Ly = b
    auxy = L.copy()
    for i in range(len(data)):
        auxy[i].append(b[i])
    
    y = solucaoInferior(auxy)
    
    #1. Ux = y:
    auxx = U.copy()
    for i in range(len(data)):
        auxx[i].append(y[i])

    x = solucaoSuperior(auxx)


    return y, x, pd.DataFrame(L), pd.DataFrame(U)

y, x, L, U = solucaoLU(data, b)

print(L,"\n\n", y,"\n\n")
print(U,"\n\n",x)

     0    1  2  3
0  1.0  0.0  0 -2
1  2.0  1.0  0  1
2  2.0  3.0  1  3 

 [-2.0, 5.0, -8.0] 


   0    1    2    3
0  1  1.0  1.0 -2.0
1  0 -1.0 -3.0  5.0
2  0  0.0  8.0 -8.0 

 [-1.0, -2.0, 1.0]


#### Matriz inversa de A
* Um método para obter a matriz A^(-1) é realizar o escalonamento da matriz [A|I] onde I é a matriz identidade. Ao terminar o escalonamento, o bloco do lado direito conterá A^(-1)

### Método de Jacobi

* O método de Jacobi é uma simplicação do algorítmo de valores próprios de Jacobi, sendo ele formulado pelo matemático alemão Carl Gustav Jakob Jacobi no final do século XVIII.
* Para resolução de sistemas lineares com poucas dimensões, dificilmente usa-se métodos iterativos, uma vez que eles levam maior tempo para obter uma aproximação do real valor se comparados com os métódos diretos, como por exemplo a eliminação gaussiana.

* Entretanto, para sistemas grandes, como os que encontramos ao fazermos a análise de determinados circuitos ou na resolução de equações diferenciais parciais, essas técnicas iterativas tornam-se mais eficientes.

In [10]:
def metodoJacobiano(data, chute, erro = 0.05):
    d = []
    B = []
    #1. escrevemos as matrizes B e d do processo iterativo:
    for linha in range(len(data)):
        pivo = data[linha][linha]
        auxl = []
        for coluna in range(len(data)+1):


            if(coluna == linha):
                data[linha][coluna] = 0
                auxl.append(0)
            elif(coluna == len(data[0])-1):
                data[linha][coluna] /= pivo
                d.append(data[linha][coluna] )
            else:
                data[linha][coluna] = (data[linha][coluna]/pivo)*(-1)
                auxl.append(data[linha][coluna])
        B.append(auxl)
                
    
    x = chute
    x0 = chute
    #2. daremos um chute inicial
    while True:
        x = np.dot(B,x) + d
        print(x)
        erroR = max(abs(x-x0))/max(abs(x))
        print("Erro relativo:",erroR)
        if(erroR > erro): break
        x0 = x.copy()
        
    return data

data = [[10,2,1,7],[1,5,1,-8],[2, 3, 10, 6]]
chute = [0.7, -1.6, 0.6]

dataf = pd.DataFrame(metodoJacobiano(data, chute))

dataf


[ 0.96 -1.86  0.94]
Erro relativo: 0.18279569892473116


,0,1,2,3
0,0.0,-0.2,-0.1,0.7
1,-0.2,0.0,-0.2,-1.6
2,-0.2,-0.3,0.0,0.6


### Método de Gauss-Seidel

In [11]:
def metodoGaussSeidel(data, chute, erro = 0.05, iteracao = 5):
    d = []
    B = []
    #1. escrevemos as matrizes B e d do processo iterativo:
    for linha in range(len(data)):
        pivo = data[linha][linha]
        auxl = []
        for coluna in range(len(data)+1):


            if(coluna == linha):
                data[linha][coluna] = 0
                auxl.append(0)
            elif(coluna == len(data[0])-1):
                data[linha][coluna] /= pivo
                d.append(data[linha][coluna] )
            else:
                data[linha][coluna] = (data[linha][coluna]/pivo)*(-1)
                auxl.append(data[linha][coluna])
        B.append(auxl)
                

    x = chute
    x0 = chute.copy()
    
    while True:
        for i in range(len(B[0])):
            x[i] = np.dot(B[i],x) + d[i]

        print(x)
        erroR = max(abs(np.array(x)-np.array(x0))) / max(abs(np.array(x)))
        print("Erro relativo:",erroR)
        if(erroR > erro): break
        x0 = x.copy()
    return data

data = [[5,1,1,5],[3,4,1,6],[3,3,6,0]]
chute = [0,0,0]


dataf = pd.DataFrame(metodoGaussSeidel(data, chute))

dataf

[1.0, 0.75, -0.875]
Erro relativo: 1.0


,0,1,2,3
0,0.00,-0.2,-0.20,1.0
1,-0.75,0.0,-0.25,1.5
2,-0.50,-0.5,0.00,0.0
